In [21]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

# Actors

In [22]:
# filepath = "dbfs:/FileStore/tables/Files/actors.csv"
filepath = "actors.csv"
df = spark.read.format("csv") \
    .option("inferSchema", "True") \
    .option("header", "True") \
    .load(filepath)

display(df)

DataFrame[imdb_title_id: string, ordering: int, imdb_name_id: string, category: string, job: string, characters: string]

**See current schema**

In [23]:
df.printSchema()

root
 |-- imdb_title_id: string (nullable = true)
 |-- ordering: integer (nullable = true)
 |-- imdb_name_id: string (nullable = true)
 |-- category: string (nullable = true)
 |-- job: string (nullable = true)
 |-- characters: string (nullable = true)



**Create new schema**

In [24]:
schema = "title STRING, order INT, name STRING, category STRING, job STRING, characters STRING"

**Read DataFrame with new schema**

In [25]:
df = spark.read.format("csv") \
    .schema(schema) \
    .option("header", "True") \
    .load(filepath)

display(df)

DataFrame[title: string, order: int, name: string, category: string, job: string, characters: string]

#### Ex. 3
**Create json by hand**

In [34]:
json = '{"title": "tt0000009","order": 1,"name": "nm0063086","category": "actress","job": "null","characters": "[Miss Geraldine Holbrook (Miss Jerry)]"}'
df_json = spark.read.json(sc.parallelize([json]))

In [35]:
df_json.show()

+--------+--------------------+----+---------+-----+---------+
|category|          characters| job|     name|order|    title|
+--------+--------------------+----+---------+-----+---------+
| actress|[Miss Geraldine H...|null|nm0063086|    1|tt0000009|
+--------+--------------------+----+---------+-----+---------+



**or write DataFrame to json and read again**

In [36]:
# df.select("title", "name").write.save("title_name.json", format="json")

In [37]:
# df_json = spark.read.format("json").json("title_name.json")

# df_json.show()


#### Ex. 4
**Damaging the data**

In [52]:
json = '"title": "tt0000009","order" 1,"name": "nm0063086","category": "actress" "job": "null","characters": "[Miss Geraldine Holbrook (Miss Jerry)]"}'

In [53]:
df_1 = spark.read.schema(schema) \
    .option("header", "True") \
    .option("badRecordsPath", "/mnt/sources/badrecords") \
    .json(sc.parallelize([json]))

df_1.show()

+-----+-----+----+--------+----+----------+
|title|order|name|category| job|characters|
+-----+-----+----+--------+----+----------+
| null| null|null|    null|null|      null|
+-----+-----+----+--------+----+----------+



In [54]:
df_2 = spark.read.schema(schema) \
    .option("header", "True") \
    .option("mode", "PERMISSIVE") \
    .json(sc.parallelize([json]))

df_2.show()

+-----+-----+----+--------+----+----------+
|title|order|name|category| job|characters|
+-----+-----+----+--------+----+----------+
| null| null|null|    null|null|      null|
+-----+-----+----+--------+----+----------+



In [55]:
df_3 = spark.read.schema(schema) \
    .option("header", "True") \
    .option("mode", "DROPMALFORMED") \
    .json(sc.parallelize([json]))

df_3.show()

+-----+-----+----+--------+---+----------+
|title|order|name|category|job|characters|
+-----+-----+----+--------+---+----------+
+-----+-----+----+--------+---+----------+



In [56]:
df_4 = spark.read.schema(schema) \
    .option("header", "True") \
    .option("mode", "FAILFAST") \
    .json(sc.parallelize([json]))

df_4.show()

23/03/19 17:37:58 ERROR Executor: Exception in task 0.0 in stage 21.0 (TID 21)
org.apache.spark.SparkException: Malformed records are detected in record parsing. Parse Mode: FAILFAST. To process malformed records as null result, try setting the option 'mode' as 'PERMISSIVE'.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.malformedRecordsDetectedInRecordParsingError(QueryExecutionErrors.scala:1417)
	at org.apache.spark.sql.catalyst.util.FailureSafeParser.parse(FailureSafeParser.scala:68)
	at org.apache.spark.sql.DataFrameReader.$anonfun$json$12(DataFrameReader.scala:430)
	at org.apache.spark.sql.DataFrameReader$$Lambda$3357/1880042285.apply(Unknown Source)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedR

Py4JJavaError: An error occurred while calling o577.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 21.0 failed 1 times, most recent failure: Lost task 0.0 in stage 21.0 (TID 21) (192.168.0.171 executor driver): org.apache.spark.SparkException: Malformed records are detected in record parsing. Parse Mode: FAILFAST. To process malformed records as null result, try setting the option 'mode' as 'PERMISSIVE'.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.malformedRecordsDetectedInRecordParsingError(QueryExecutionErrors.scala:1417)
	at org.apache.spark.sql.catalyst.util.FailureSafeParser.parse(FailureSafeParser.scala:68)
	at org.apache.spark.sql.DataFrameReader.$anonfun$json$12(DataFrameReader.scala:430)
	at org.apache.spark.sql.DataFrameReader$$Lambda$3357/1880042285.apply(Unknown Source)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at org.apach

In [57]:
spark.stop()